In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import hydrant.ObjectiveFunctions.ObjectiveFunctions as obj

In [2]:
# Example usage
# Create example data for observation
time_obs = pd.date_range(start='1950-01-01', end='2020-12-31', freq='D')
ids_obs = np.arange(1, 101)
data_obs = np.random.rand(len(time_obs), len(ids_obs))
# add np.nan values to emulate missing values in observation
data_obs[:,1] = np.nan
data_obs[:,-1] = np.nan
data_obs[1,:] = np.nan
data_obs[-1,:] = np.nan
# convert to dataframe
observation = xr.Dataset({'Discharge_obs': (['time', 'COMID'], data_obs)},
                         coords={'time': time_obs, 'COMID': ids_obs})
observation

<xarray.Dataset>
Dimensions:        (time: 25933, COMID: 100)
Coordinates:
  * time           (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2020-12-31
  * COMID          (COMID) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
Data variables:
    Discharge_obs  (time, COMID) float64 0.6478 nan 0.1692 ... nan nan nan

In [3]:
# Create example data for simulation
time_sim = pd.date_range(start='1950-01-01 12:00:00', end='2100-12-31 12:00:00', freq='D')
ids_sim = np.arange(50, 151)
data_sim = np.random.rand(len(time_sim), len(ids_sim))
simulation = xr.Dataset({'Discharge_sim': (['time', 'SegId'], data_sim)},
                        coords={'time': time_sim, 'SegId': ids_sim})
simulation

<xarray.Dataset>
Dimensions:        (time: 55152, SegId: 101)
Coordinates:
  * time           (time) datetime64[ns] 1950-01-01T12:00:00 ... 2100-12-31T1...
  * SegId          (SegId) int64 50 51 52 53 54 55 ... 145 146 147 148 149 150
Data variables:
    Discharge_sim  (time, SegId) float64 0.4648 0.6273 0.8803 ... 0.6583 0.4381

In [4]:
# Call the ObjectiveFunction with the example datasets
Objective = obj.ObjectiveFunction(observation,
                                  simulation,
                                  info_obs={'var': 'Discharge_obs', 'var_id': 'COMID', 'dim_id': 'COMID', 'var_time': 'time', 'dim_time': 'time'},
                                  info_sim={'var': 'Discharge_sim', 'var_id': 'SegId', 'dim_id': 'SegId', 'var_time': 'time', 'dim_time': 'time'},
                                  TimeStep='daily')

# Print the result
Objective

/home/shg096/FHIMP/virtual_env/fhimp_venv/lib/python3.10/site-packages/hydrant/ObjectiveFucntions/ObjectiveFucntions.py:18: UserWarning: rename 'time' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds_renamed = ds_renamed.rename({var_time: 'time'})


<xarray.Dataset>
Dimensions:            (time: 25933, ID: 51)
Coordinates:
  * time               (time) datetime64[ns] 1950-01-01 ... 2020-12-31
  * ID                 (ID) int64 50 51 52 53 54 55 56 ... 94 95 96 97 98 99 100
Data variables:
    Discharge_obs_obs  (time, ID) float64 0.5288 0.3499 0.2056 ... nan nan nan
    Discharge_sim_sim  (time, ID) float64 0.4648 0.6273 0.8803 ... 0.493 0.854
    KGE                (ID) float64 0.008605 0.01262 0.007382 ... 0.0009506 nan
    NSE                (ID) float64 -0.9781 -0.9709 -0.9949 ... -1.006 nan
    RMSE               (ID) float64 0.4057 0.4058 0.407 ... 0.4094 0.4078 nan